In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
        }
head = pd.DataFrame([head])
head.to_csv("./Adversarial_Training_pgd.csv", mode='a', index=False)



In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Adversarial_Training_pgd.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def1/y_pred_{attack_name}{eps}_Def1.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# PGD attack function
def pgd_attack(model, images, labels, eps=0.3, alpha=2/255, iters=10):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()

    ori_images = images.data

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        adv_images = images + alpha*images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# # Adversarial training loop
# for epoch in range(10):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         # Generate adversarial examples
#         adv_inputs = pgd_attack(model, inputs, labels, eps=0.3, alpha=2/255, iters=10)

#         optimizer.zero_grad()
#         outputs = model(adv_inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break


In [12]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/Adversarial_Training/Adversarial_Training_pgd.pt"))
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")


fc1.weight: torch.Size([50, 42])
fc1.bias: torch.Size([50])
fc2.weight: torch.Size([30, 50])
fc2.bias: torch.Size([30])
fc3.weight: torch.Size([20, 30])
fc3.bias: torch.Size([20])
fc4.weight: torch.Size([5, 20])
fc4.bias: torch.Size([5])


/tmp/ipykernel_9590/821411865.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/A

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Adversarial_Training_pgd.csv", mode='a', index=False, header=False)

In [ ]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 FGSM
Accuracy: 0.9909701708531167

macro
Precision: 0.551573068281288
Recall: 0.4811213728844013
F1 Score: 0.4913064698688213

weighted
Precision: 0.9916349305363
Recall: 0.9909701708531167
F1 Score: 0.9912722880768008

start XGB 100 PGD
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.839622641509434

macro
Precision: 0.4946236559139785
Recall: 0.6043956043956044
F1 Score: 0.5163812033377251

weighted
Precision: 0.9269628727936701
Recall: 0.839622641509434
F1 Score: 0.8623287841499491

start XGB 100 AutoPGD
Accuracy: 0.9921522784419251

macro
Precision: 0.347686473524785
Recall: 0.3758651493907904
F1 Score: 0.34783117720601137

weighted
Precision: 0.9933546472920541
Recall: 0.9921522784419251
F1 Score: 0.9926992560927835

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.37766728817070644

macro
Precision: 0.34492887318150584
Recall: 0.6902525602915767
F1 Score: 0.26696801732138636

weighted
Precision: 0.8107917766448276
Recall: 0.37766728817070644
F1 Score: 0.3772923897016547

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9988193624557261

macro
Precision: 0.3351351351351351
Recall: 0.6662880732375086
F1 Score: 0.33672815844861415

weighted
Precision: 0.9999487026166869
Recall: 0.9988193624557261
F1 Score: 0.9993805532689097

start XGB 100 VNIFGSM
Accuracy: 0.9873455533526871

macro
Precision: 0.6831340396437999
Recall: 0.5674720250644947
F1 Score: 0.6039927841832073

weighted
Precision: 0.9869932312238994
Recall: 0.9873455533526871
F1 Score: 0.9871102888323539

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM
Accuracy: 0.9982495904490474

macro
Precision: 0.44199949659118654
Recall: 0.41778282594614913
F1 Score: 0.37909229885125945

weighted
Precision: 0.9996222034456566
Recall: 0.9982495904490474
F1 Score: 0.9988828074737105

start XGB 50 FGSM
Accuracy: 0.9890585478433822

macro
Precision: 0.579088949136439
Recall: 0.49526034850329753
F1 Score: 0.5091762422665577

weighted

...

Accuracy: 0.7424242424242424

macro
Precision: 0.36875
Recall: 0.39055023923444976
F1 Score: 0.3767995570321152

weighted
Precision: 0.7465909090909091
Recall: 0.7424242424242424
F1 Score: 0.7405282056444847

start RF 20 AutoPGD


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9820997953297352

macro
Precision: 0.42636750803999224
Recall: 0.3131062120824607
F1 Score: 0.340778796559791

weighted
Precision: 0.981188633721774
Recall: 0.9820997953297352
F1 Score: 0.9788935136925537

start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input20/x_test_adv_ZOO_Def1.npy not found
start RF 20 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8340753932917779

macro
Precision: 0.5284885528347673
Recall: 0.5491250935278713
F1 Score: 0.47783714347366646

weighted
Precision: 0.8901306524371455
Recall: 0.8340753932917779
F1 Score: 0.8516432694967151

start RF 20 SINIFGSM
Accuracy: 0.9643628509719222

macro
Precision: 0.8747086815604913
Recall: 0.6560734463276836
F1 Score: 0.7355493083388303

weighted
Precision: 0.9587345067287141
Recall: 0.9643628509719222
F1 Score: 0.9586437742762928

start RF 20 VNIFGSM


In [16]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9981456352406569

macro
Precision: 0.41923889546839366
Recall: 0.36641392248533544
F1 Score: 0.3872278533836494

weighted
Precision: 0.9977067271717284
Recall: 0.9981456352406569
F1 Score: 0.9979045686455741

start XGB 100 FGSM
Accuracy: 0.988504761276418

macro
Precision: 0.5467023886917836
Recall: 0.490799302553459
F1 Score: 0.48978558295092095

weighted
Precision: 0.989308634654993
Recall: 0.988504761276418
F1 Score: 0.9888844888556841

start XGB 100 PGD
Accuracy: 0.9981456352406569

macro
Precision: 0.41923889546839366
Recall: 0.36641392248533544
F1 Score: 0.3872278533836494

weighted
Precision: 0.9977067271717284
Recall: 0.9981456352406569
F1 Score: 0.9979045686455741

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.7976190476190477

macro
Precision: 0.3752525252525253
Recall: 0.4083333333333333
F1 Score: 0.3833333333333333

weighted
Precision: 0.8506493506493507
Recall: 0.7976190476190477
F1 Score: 0.8134920634920636

start XGB 100 AutoPGD
Accuracy: 0.9912720830880407

macro
Precision: 0.426403039830115
Recall: 0.35930318717725396
F1 Score: 0.3649963656410887

weighted
Precision: 0.9920258572671788
Recall: 0.9912720830880407
F1 Score: 0.9915212120129763

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.08973334063406888

macro
Precision: 0.2323822825555685
Recall: 0.657020406018676
F1 Score: 0.08239078986523739

weighted
Precision: 0.8753750097099915
Recall: 0.08973334063406888
F1 Score: 0.04426909564038427

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9878678704748676

macro
Precision: 0.5005241414455674
Recall: 0.41624236487645283
F1 Score: 0.25448614143989146

weighted
Precision: 0.9976403991864039
Recall: 0.9878678704748676
F1 Score: 0.9913204069758281

start XGB 100 VNIFGSM
Accuracy: 0.9855150155416867

macro
Precision: 0.6797244571134129
Recall: 0.5775643181801839
F1 Score: 0.6178678470338312

weighted
Precision: 0.9853978047184777
Recall: 0.9855150155416867
F1 Score: 0.9854133433409369

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM
Accuracy: 0.9961519319243982

macro
Precision: 0.34586173930557723
Recall: 0.2578536961802228
F1 Score: 0.28265729135108353

weighted
Precision: 0.9953443544976296
Recall: 0.9961519319243982
F1 Score: 0.9953671886468399

start XGB 50 FGSM
Accuracy: 0.9910822468860563

macro
Precision: 0.6138016650643534
Recall: 0.5406348279952343
F1 Score: 0.56025945287779

...

Accuracy: 0.05925925925925926

macro
Precision: 0.26346153846153847
Recall: 0.19642857142857142
F1 Score: 0.02911686398912366

weighted
Precision: 0.9369230769230769
Recall: 0.05925925925925926
F1 Score: 0.021412791027020903

start RF 1 AutoPGD
Accuracy: 0.994565634546599

macro
Precision: 0.5805658174577255
Recall: 0.45874331575076566
F1 Score: 0.5026123394199977

weighted
Precision: 0.9942399268053895
Recall: 0.994565634546599
F1 Score: 0.994308127225949

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input1/x_test_adv_ZOO_Def1.npy not found
start RF 1 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.10720562390158173

macro
Precision: 0.2364630467571644
Recall: 0.381621975147155
F1 Score: 0.10551837360424673

weighted
Precision: 0.959919495715009
Recall: 0.10720562390158173
F1 Score: 0.16448039105218776

start RF 1 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8607395751376868

macro
Precision: 0.3333333333333333
Recall: 0.28713910761154854
F1 Score: 0.308516638465877

weighted
Precision: 0.999213217938631
Recall: 0.8607395751376868
F1 Score: 0.9248217093272946

start RF 1 VNIFGSM
Accuracy: 0.9907692233689208

macro
Precision: 0.682564229421627
Recall: 0.5156733987890434
F1 Score: 0.5607525844851832

weighted
Precision: 0.9912533883550849
Recall: 0.9907692233689208
F1 Score: 0.9909175780886719



In [15]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["2", "3"]

for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9708250769425355

macro
Precision: 0.7349962435290545
Recall: 0.5565723623391451
F1 Score: 0.610060702163068

weighted
Precision: 0.9698382497671121
Recall: 0.9708250769425355
F1 Score: 0.9687734123770182

start XGB 100 FGSM
Accuracy: 0.9742617560212535

macro
Precision: 0.525596365693372
Recall: 0.6049118876207985
F1 Score: 0.5447288463471424

weighted
Precision: 0.9765764051682794
Recall: 0.9742617560212535
F1 Score: 0.9749465200465125

start XGB 100 PGD
Accuracy: 0.9708250769425355

macro
Precision: 0.7349962435290545
Recall: 0.5565723623391451
F1 Score: 0.610060702163068

weighted
Precision: 0.9698382497671121
Recall: 0.9708250769425355
F1 Score: 0.9687734123770182

start XGB 100 DF
Accuracy: 0.009146581933317558

macro
Precision: 0.014897056200084741
Recall: 0.041415049610021216
F1 Scor

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7086599120627031

macro
Precision: 0.5221140276164666
Recall: 0.45386406788692585
F1 Score: 0.45620043615438066

weighted
Precision: 0.7566106852118822
Recall: 0.7086599120627031
F1 Score: 0.7259195078400604

start XGB 10 VNIFGSM
Accuracy: 0.9843460638037668

macro
Precision: 0.7526179646676235
Recall: 0.6844012847977777
F1 Score: 0.7067864497854413

weighted
Precision: 0.9845147617067337
Recall: 0.9843460638037668
F1 Score: 0.9843953293623693

start XGB 1 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input1/x_test_adv_baseline_Def1.npy not found
start XGB 1 BIM
Accuracy: 0.9790395249046505

macro
Precision: 0.6608438057514634
Recall: 0.40802971388156706
F1 Score: 0.47698303568823663

weighted
Precision: 0.9757838958191452
Recall: 0.9790395249046505
F1 Score: 0.9759708246386672

start XGB 1 FGSM
Accuracy: 0.9719970384104738

macro
Precision: 0.43225274563991534
Recall: 0.4569422500250435
F1 Score: 0.431881561721668

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9264275063590913

macro
Precision: 0.40901433616392036
Recall: 0.29075728025998115
F1 Score: 0.32377956493625115

weighted
Precision: 0.9102968548906689
Recall: 0.9264275063590913
F1 Score: 0.9124721652135371

start RF 50 VNIFGSM
Accuracy: 0.9841055699958959

macro
Precision: 0.7380021645279652
Recall: 0.6660092863815712
F1 Score: 0.6904627411416802

weighted
Precision: 0.9841514419763062
Recall: 0.9841055699958959
F1 Score: 0.9840762207605319

start RF 20 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input20/x_test_adv_baseline_Def1.npy not found
start RF 20 BIM
Accuracy: 0.9806186348720765

macro
Precision: 0.6661668845160975
Recall: 0.4285776572153496
F1 Score: 0.49418519082627865

weighted
Precision: 0.9781797736066635
Recall: 0.9806186348720765
F1 Score: 0.978422453076088

start RF 20 FGSM
Accuracy: 0.9723290196728953

macro
Precision: 0.43175155661752385
Recall: 0.45588972789756654
F1 Score: 0.4314313867783254

...

Accuracy: 0.563974076301196

macro
Precision: 0.3718932616400397
Recall: 0.3284197087745684
F1 Score: 0.30845921397086246

weighted
Precision: 0.7432781132696801
Recall: 0.563974076301196
F1 Score: 0.5816051637895718

start RF 20 VNIFGSM
Accuracy: 0.9838700816849151

macro
Precision: 0.7443974181508632
Recall: 0.6685955138653692
F1 Score: 0.6930911708196502

weighted
Precision: 0.9838767741010963
Recall: 0.9838700816849151
F1 Score: 0.9838061274641784

start RF 10 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input10/x_test_adv_baseline_Def1.npy not found
start RF 10 BIM
Accuracy: 0.9866934421474703

macro
Precision: 0.6435158071789915
Recall: 0.44060531734660896
F1 Score: 0.48949452444010777

weighted
Precision: 0.9852474747562098
Recall: 0.9866934421474703
F1 Score: 0.9852092348480017

start RF 10 FGSM
Accuracy: 0.9719547656158346

macro
Precision: 0.4329352287718649
Recall: 0.4605202890075707
F1 Score: 0.4331768651231693

we

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.491365568544102

macro
Precision: 0.39007235838818466
Recall: 0.313518778754041
F1 Score: 0.30414469461605614

weighted
Precision: 0.6541204272027398
Recall: 0.491365568544102
F1 Score: 0.5047409547840641

start RF 10 VNIFGSM
Accuracy: 0.9840162065271576

macro
Precision: 0.7528401901125793
Recall: 0.673441423867186
F1 Score: 0.6999644355919021

weighted
Precision: 0.9840536558480841
Recall: 0.9840162065271576
F1 Score: 0.9839735788492808

start RF 1 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input1/x_test_adv_baseline_Def1.npy not found
start RF 1 BIM
Accuracy: 0.9706524662471088

macro
Precision: 0.6713397410909234
Recall: 0.4015674399890859
F1 Score: 0.4727192706112038

weighted
Precision: 0.9668265792135238
Recall: 0.9706524662471088
F1 Score: 0.966240864984321

start RF 1 FGSM
Accuracy: 0.9718802083879285

macro
Precision: 0.43233764320383417
Recall: 0.4554370242815528
F1 Score: 0.4318662333814281

weighted


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4333236490412551

macro
Precision: 0.33779425659815066
Recall: 0.33547441759692953
F1 Score: 0.28681012331911476

weighted
Precision: 0.7237505236521791
Recall: 0.4333236490412551
F1 Score: 0.4928531512852802

start RF 1 VNIFGSM
Accuracy: 0.9834676616704714

macro
Precision: 0.7491674283691541
Recall: 0.6509853684243994
F1 Score: 0.6790699910355439

weighted
Precision: 0.9834998778141466
Recall: 0.9834676616704714
F1 Score: 0.9834189425587369



In [ ]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_baseline_Def1.npy not found
start BIM
Accuracy: 0.9759979515877051

macro
Precision: 0.5105386251540244
Recall: 0.47335850293834403
F1 Score: 0.4189458557909834

weighted
Precision: 0.9753772599902423
Recall: 0.9759979515877051
F1 Score: 0.9731879227803308

start FGSM
Accuracy: 0.9922585190850247

macro
Precision: 0.47298415653036746
Recall: 0.4761282721368893
F1 Score: 0.4644923947795302

weighted
Precision: 0.992188949534287
Recall: 0.9922585190850247
F1 Score: 0.9921590275005343

start PGD
Accuracy: 0.9759979515877051

macro
Precision: 0.5105386251540244
Recall: 0.47335850293834403
F1 Score: 0.4189458557909834

weighted
Precision: 0.9753772599902423
Recall: 0.9759979515877051
F1 Score: 0.9731879227803308

start DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5912408759124088

macro
Precision: 0.241025641025641
Recall: 0.3140625
F1 Score: 0.21335873822377477

weighted
Precision: 0.9462848586936178
Recall: 0.5912408759124088
F1 Score: 0.6985294060834205

start AutoPGD
Accuracy: 0.970725542872988

macro
Precision: 0.6361993294788149
Recall: 0.561591702769044
F1 Score: 0.5148293923838458

weighted
Precision: 0.9692288591577242
Recall: 0.970725542872988
F1 Score: 0.9671195940024093

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_ZOO_Def1.npy not found
start CaFA
Accuracy: 0.1458616279223091

macro
Precision: 0.23710620272112543
Recall: 0.6146470926608638
F1 Score: 0.09974278734236805

weighted
Precision: 0.9087120120778885
Recall: 0.1458616279223091
F1 Score: 0.14368615465577686

start SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9837993453657978

macro
Precision: 0.5535654067233517
Recall: 0.4618255139954751
F1 Score: 0.49175360509789845

weighted
Precision: 0.9831442565351244
Recall: 0.9837993453657978
F1 Score: 0.9816761650491761

start VNIFGSM
Accuracy: 0.9873828114446276

macro
Precision: 0.7438209688698094
Recall: 0.764795156174223
F1 Score: 0.7203974804724228

weighted
Precision: 0.9874838385178429
Recall: 0.9873828114446276
F1 Score: 0.9873498123170457

start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_baseline_Def1.npy not found
start BIM


...

Accuracy: 0.6708860759493671

macro
Precision: 0.24
Recall: 0.33108108108108103
F1 Score: 0.22503401360544215

weighted
Precision: 0.9481012658227849
Recall: 0.6708860759493671
F1 Score: 0.7607078274347713

start AutoPGD
Accuracy: 0.9484768475365529

macro
Precision: 0.5567104546472652
Recall: 0.4017313249295399
F1 Score: 0.42130631283599873

weighted
Precision: 0.9442075739362945
Recall: 0.9484768475365529
F1 Score: 0.9406735949639362

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_ZOO_Def1.npy not found
start CaFA
Accuracy: 0.137247289563742

macro
Precision: 0.23442428043595606
Recall: 0.6544228090633084
F1 Score: 0.09297198600137278

weighted
Precision: 0.9086737798156534
Recall: 0.137247289563742
F1 Score: 0.12794301045017625

start SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9559051606912283

macro
Precision: 0.4519054643533968
Recall: 0.2892990034518494
F1 Score: 0.32927449961962807

weighted
Precision: 0.9455119511674767
Recall: 0.9559051606912283
F1 Score: 0.9459983666138844

start VNIFGSM
Accuracy: 0.9852855073145089

macro
Precision: 0.7506154890088477
Recall: 0.6987665120891907
F1 Score: 0.7145383885116812

weighted
Precision: 0.9853915871213839
Recall: 0.9852855073145089
F1 Score: 0.9853048163781241

start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_baseline_Def1.npy not found
start BIM
Accuracy: 0.9830625007242374

macro
Precision: 0.7041689280996796
Recall: 0.6825931999977315
F1 Score: 0.6222602018026882

weighted
Precision: 0.981420066735257
Recall: 0.9830625007242374
F1 Score: 0.980978370653585

start FGSM
Accuracy: 0.9926143505353148

macro
Precision: 0.5195824790623153
Recall: 0.5143806674664042
F1 Score: 0.49772060769272414

weighted
Precision: 0.992456823248

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.5217391304347826

macro
Precision: 0.225531914893617
Recall: 0.3007633587786259
F1 Score: 0.17929317115218849

weighted
Precision: 0.9548257786000616
Recall: 0.5217391304347826
F1 Score: 0.6459068138567436

start AutoPGD
Accuracy: 0.977720447445329

macro
Precision: 0.7056089436932267
Recall: 0.6346197925480546
F1 Score: 0.5883041780309384

weighted
Precision: 0.9756451357274036
Recall: 0.977720447445329
F1 Score: 0.9755249658244212

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_ZOO_Def1.npy not found
start CaFA
Accuracy: 0.14393317920799464

macro
Precision: 0.23679012860622164
Recall: 0.6437218193046896
F1 Score: 0.09818854101760596

weighted
Precision: 0.9130411401654813
Recall: 0.14393317920799464
F1 Score: 0.14278277653057472

start SINIFGSM
Accuracy: 0.9922667350931776

macro
Precision: 0.7458550412747273
Recall: 0.703544642961105
F1 Score: 0.7059898911268984

weighted
Precision: 0.992100926390985

In [18]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 FGSM
Accuracy: 0.9909701708531167

macro
Precision: 0.551573068281288
Recall: 0.4811213728844013
F1 Score: 0.4913064698688213

weighted
Precision: 0.9916349305363
Recall: 0.9909701708531167
F1 Score: 0.9912722880768008

start XGB 100 PGD
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.839622641509434

macro
Precision: 0.4946236559139785
Recall: 0.6043956043956044
F1 Score: 0.5163812033377251

weighted
Precision: 0.9269628727936701
Recall: 0.839622641509434
F1 Score: 0.8623287841499491

start XGB 100 AutoPGD
Accuracy: 0.9921522784419251

macro
Precision: 0.347686473524785
Recall: 0.3758651493907904
F1 Score: 0.34783117720601137

weighted
Precision: 0.9933546472920541
Recall: 0.9921522784419251
F1 Score: 0.9926992560927835

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.37766728817070644

macro
Precision: 0.34492887318150584
Recall: 0.6902525602915767
F1 Score: 0.26696801732138636

weighted
Precision: 0.8107917766448276
Recall: 0.37766728817070644
F1 Score: 0.3772923897016547

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9988193624557261

macro
Precision: 0.3351351351351351
Recall: 0.6662880732375086
F1 Score: 0.33672815844861415

weighted
Precision: 0.9999487026166869
Recall: 0.9988193624557261
F1 Score: 0.9993805532689097

start XGB 100 VNIFGSM
Accuracy: 0.9873455533526871

macro
Precision: 0.6831340396437999
Recall: 0.5674720250644947
F1 Score: 0.6039927841832073

weighted
Precision: 0.9869932312238994
Recall: 0.9873455533526871
F1 Score: 0.9871102888323539

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM
Accuracy: 0.9955142677206887

macro
Precision: 0.34161485325678287
Recall: 0.2614247695603713
F1 Score: 0.2854808729064312

weighted
Precision: 0.9942280064123304
Recall: 0.9955142677206887
F1 Score: 0.9945201765064552

start XGB 50 FGSM
Accuracy: 0.9850913156913902

macro
Precision: 0.3995020832936773
Recall: 0.39495686465550073
F1 Score: 0.39544752862137084



...

Accuracy: 0.999908119059323

macro
Precision: 0.33333180185064365
Recall: 0.333307300041347
F1 Score: 0.33331955049572276

weighted
Precision: 0.9999816234741508
Recall: 0.999908119059323
F1 Score: 0.9999448699159378

start RF 10 VNIFGSM
Accuracy: 0.9715295601380654

macro
Precision: 0.7315100688167077
Recall: 0.5897754486861477
F1 Score: 0.6401883880780951

weighted
Precision: 0.9717843199832994
Recall: 0.9715295601380654
F1 Score: 0.97093470611833

start RF 1 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_baseline_Def1.npy not found
start RF 1 BIM
Accuracy: 0.9984188142924648

macro
Precision: 0.3347326986491936
Recall: 0.31718505544595565
F1 Score: 0.32099753874529996

weighted
Precision: 0.9985870490365556
Recall: 0.9984188142924648
F1 Score: 0.9984795232891498

start RF 1 FGSM
Accuracy: 0.9810860384763043

macro
Precision: 0.5419323723005515
Recall: 0.5127869389672907
F1 Score: 0.5145837211370915

weighted
Precis

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9962092912690597

macro
Precision: 0.3772540137724204
Recall: 0.3234449892973409
F1 Score: 0.33842365138105535

weighted
Precision: 0.9966550187679479
Recall: 0.9962092912690597
F1 Score: 0.9961968962223666

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_ZOO_Def1.npy not found
start RF 1 CaFA
Accuracy: 0.32472006890611543

macro
Precision: 0.19369203965925005
Recall: 0.5222172708855133
F1 Score: 0.16425526375671717

weighted
Precision: 0.4125276482591745
Recall: 0.32472006890611543
F1 Score: 0.21394097676757343

start RF 1 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.999983777633064

macro
Precision: 0.24999710313533458
Recall: 0.25
F1 Score: 0.24999855155927542

weighted
Precision: 0.9999721903623782
Recall: 0.999983777633064
F1 Score: 0.9999779839641542

start RF 1 VNIFGSM
Accuracy: 0.9854219920300936

macro
Precision: 0.7055085471045066
Recall: 0.5601478159524876
F1 Score: 0.6050011063677444

weighted
Precision: 0.9856734667541734
Recall: 0.9854219920300936
F1 Score: 0.985271239904221



In [19]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 FGSM
Accuracy: 0.9909701708531167

macro
Precision: 0.551573068281288
Recall: 0.4811213728844013
F1 Score: 0.4913064698688213

weighted
Precision: 0.9916349305363
Recall: 0.9909701708531167
F1 Score: 0.9912722880768008

start XGB 100 PGD
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.839622641509434

macro
Precision: 0.4946236559139785
Recall: 0.6043956043956044
F1 Score: 0.5163812033377251

weighted
Precision: 0.9269628727936701
Recall: 0.839622641509434
F1 Score: 0.8623287841499491

start XGB 100 AutoPGD
Accuracy: 0.9921522784419251

macro
Precision: 0.347686473524785
Recall: 0.3758651493907904
F1 Score: 0.34783117720601137

weighted
Precision: 0.9933546472920541
Recall: 0.9921522784419251
F1 Score: 0.9926992560927835

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.37766728817070644

macro
Precision: 0.34492887318150584
Recall: 0.6902525602915767
F1 Score: 0.26696801732138636

weighted
Precision: 0.8107917766448276
Recall: 0.37766728817070644
F1 Score: 0.3772923897016547

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9988193624557261

macro
Precision: 0.3351351351351351
Recall: 0.6662880732375086
F1 Score: 0.33672815844861415

weighted
Precision: 0.9999487026166869
Recall: 0.9988193624557261
F1 Score: 0.9993805532689097

start XGB 100 VNIFGSM
Accuracy: 0.9873455533526871

macro
Precision: 0.6831340396437999
Recall: 0.5674720250644947
F1 Score: 0.6039927841832073

weighted
Precision: 0.9869932312238994
Recall: 0.9873455533526871
F1 Score: 0.9871102888323539

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM
Accuracy: 0.9843753096598004

macro
Precision: 0.6141314032789499
Recall: 0.4472339166177253
F1 Score: 0.5049070810361219

weighted
Precision: 0.9820920415995884
Recall: 0.9843753096598004
F1 Score: 0.9827996352924708

start XGB 50 FGSM
Accuracy: 0.9933607624184327

macro
Precision: 0.7025632419789296
Recall: 0.5231864483891027
F1 Score: 0.55510501288476

...

Accuracy: 0.940311773553945

macro
Precision: 0.3807212233448075
Recall: 0.29650580642820595
F1 Score: 0.32216974936510845

weighted
Precision: 0.942506448947674
Recall: 0.940311773553945
F1 Score: 0.9318852729780336

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input1/x_test_adv_ZOO_Def1.npy not found
start RF 1 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.11304397702411485

macro
Precision: 0.24121011193963549
Recall: 0.39673831525486136
F1 Score: 0.1062361224680826

weighted
Precision: 0.9746360240441309
Recall: 0.11304397702411485
F1 Score: 0.17397566143300283

start RF 1 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.894640283055956

macro
Precision: 0.37785900371650893
Recall: 0.2543204217281687
F1 Score: 0.2739926429934942

weighted
Precision: 0.906111077988363
Recall: 0.894640283055956
F1 Score: 0.8681022034624816

start RF 1 VNIFGSM
Accuracy: 0.9960482641554314

macro
Precision: 0.7140392382075605
Recall: 0.5165600194056389
F1 Score: 0.5481049059656293

weighted
Precision: 0.9951956196222287
Recall: 0.9960482641554314
F1 Score: 0.9952564387262377



In [20]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]




base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 FGSM
Accuracy: 0.9909701708531167

macro
Precision: 0.551573068281288
Recall: 0.4811213728844013
F1 Score: 0.4913064698688213

weighted
Precision: 0.9916349305363
Recall: 0.9909701708531167
F1 Score: 0.9912722880768008

start XGB 100 PGD
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.839622641509434

macro
Precision: 0.4946236559139785
Recall: 0.6043956043956044
F1 Score: 0.5163812033377251

weighted
Precision: 0.9269628727936701
Recall: 0.839622641509434
F1 Score: 0.8623287841499491

start XGB 100 AutoPGD
Accuracy: 0.9921522784419251

macro
Precision: 0.347686473524785
Recall: 0.3758651493907904
F1 Score: 0.34783117720601137

weighted
Precision: 0.9933546472920541
Recall: 0.9921522784419251
F1 Score: 0.9926992560927835

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.37766728817070644

macro
Precision: 0.34492887318150584
Recall: 0.6902525602915767
F1 Score: 0.26696801732138636

weighted
Precision: 0.8107917766448276
Recall: 0.37766728817070644
F1 Score: 0.3772923897016547

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9988193624557261

macro
Precision: 0.3351351351351351
Recall: 0.6662880732375086
F1 Score: 0.33672815844861415

weighted
Precision: 0.9999487026166869
Recall: 0.9988193624557261
F1 Score: 0.9993805532689097

start XGB 100 VNIFGSM
Accuracy: 0.9873455533526871

macro
Precision: 0.6831340396437999
Recall: 0.5674720250644947
F1 Score: 0.6039927841832073

weighted
Precision: 0.9869932312238994
Recall: 0.9873455533526871
F1 Score: 0.9871102888323539

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM
Accuracy: 0.9950511727855478

macro
Precision: 0.5762124389488086
Recall: 0.41110227771672686
F1 Score: 0.46955096581171835

weighted
Precision: 0.9946665790370052
Recall: 0.9950511727855478
F1 Score: 0.9945072315421772

start XGB 50 FGSM
Accuracy: 0.9941576729828988

macro
Precision: 0.6622503381825158
Recall: 0.5243986486812667
F1 Score

...

Accuracy: 0.9763461463768304

macro
Precision: 0.30062827351731436
Recall: 0.322888887011189
F1 Score: 0.3064175553962222

weighted
Precision: 0.9842403252165837
Recall: 0.9763461463768304
F1 Score: 0.9802161688410991

start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_ZOO_Def1.npy not found
start RF 1 CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.3939879311629293

macro
Precision: 0.3192432118887837
Recall: 0.46671002587967686
F1 Score: 0.27390886530550534

weighted
Precision: 0.9313500074888836
Recall: 0.3939879311629293
F1 Score: 0.5015221121803686

start RF 1 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9990865384615385

macro
Precision: 0.3333333333333333
Recall: 0.49978363304163864
F1 Score: 0.37489176968589155

weighted
Precision: 0.9999198717948717
Recall: 0.9990865384615385
F1 Score: 0.9994950049859948

start RF 1 VNIFGSM
Accuracy: 0.9984525595268884

macro
Precision: 0.7280343435264001
Recall: 0.5437202640971628
F1 Score: 0.5708106943371011

weighted
Precision: 0.998659010033458
Recall: 0.9984525595268884
F1 Score: 0.998388232129034



In [ ]:
epsilon = 0

Def = "Def1"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 FGSM
Accuracy: 0.9909701708531167

macro
Precision: 0.551573068281288
Recall: 0.4811213728844013
F1 Score: 0.4913064698688213

weighted
Precision: 0.9916349305363
Recall: 0.9909701708531167
F1 Score: 0.9912722880768008

start XGB 100 PGD
Accuracy: 0.9967219064681535

macro
Precision: 0.4356680366013605
Recall: 0.4325379023414184
F1 Score: 0.43404471464144034

weighted
Precision: 0.9968004809501121
Recall: 0.9967219064681535
F1 Score: 0.9967609618655033

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.839622641509434

macro
Precision: 0.4946236559139785
Recall: 0.6043956043956044
F1 Score: 0.5163812033377251

weighted
Precision: 0.9269628727936701
Recall: 0.839622641509434
F1 Score: 0.8623287841499491

start XGB 100 AutoPGD
Accuracy: 0.9921522784419251

macro
Precision: 0.347686473524785
Recall: 0.3758651493907904
F1 Score: 0.34783117720601137

weighted
Precision: 0.9933546472920541
Recall: 0.9921522784419251
F1 Score: 0.9926992560927835

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.37766728817070644

macro
Precision: 0.34492887318150584
Recall: 0.6902525602915767
F1 Score: 0.26696801732138636

weighted
Precision: 0.8107917766448276
Recall: 0.37766728817070644
F1 Score: 0.3772923897016547

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9988193624557261

macro
Precision: 0.3351351351351351
Recall: 0.6662880732375086
F1 Score: 0.33672815844861415

weighted
Precision: 0.9999487026166869
Recall: 0.9988193624557261
F1 Score: 0.9993805532689097

start XGB 100 VNIFGSM
Accuracy: 0.9873455533526871

macro
Precision: 0.6831340396437999
Recall: 0.5674720250644947
F1 Score: 0.6039927841832073

weighted
Precision: 0.9869932312238994
Recall: 0.9873455533526871
F1 Score: 0.9871102888323539

start XGB 50 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL/WUSTL_Input50/x_test_adv_baseline_Def1.npy not found
start XGB 50 BIM


...

Accuracy: 0.896457765667575

macro
Precision: 0.5592805755395684
Recall: 0.4215189873417722
F1 Score: 0.47389766507876746

weighted
Precision: 0.9847509458373355
Recall: 0.896457765667575
F1 Score: 0.929538183261713

start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_ExcludeCaFA/WUSTL_Input100/x_test_adv_baseline_Def1.npy not found
start XGB 100 BIM
Accuracy: 0.9981456352406569

macro
Precision: 0.41923889546839366
Recall: 0.36641392248533544
F1 Score: 0.3872278533836494

weighted
Precision: 0.9977067271717284
Recall: 0.9981456352406569
F1 Score: 0.9979045686455741

start XGB 100 FGSM
Accuracy: 0.988504761276418

macro
Precision: 0.5467023886917836
Recall: 0.490799302553459
F1 Score: 0.48978558295092095

weighted
Precision: 0.989308634654993
Recall: 0.988504761276418
F1 Score: 0.9888844888556841

start XGB 100 PGD
Accuracy: 0.9981456352406569

macro
Precision: 0.41923889546839366
Recall: 0.36641392248533544
F1 Score: 0

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.7976190476190477

macro
Precision: 0.3752525252525253
Recall: 0.4083333333333333
F1 Score: 0.3833333333333333

weighted
Precision: 0.8506493506493507
Recall: 0.7976190476190477
F1 Score: 0.8134920634920636

start XGB 100 AutoPGD
Accuracy: 0.9912720830880407

macro
Precision: 0.426403039830115
Recall: 0.35930318717725396
F1 Score: 0.3649963656410887

weighted
Precision: 0.9920258572671788
Recall: 0.9912720830880407
F1 Score: 0.9915212120129763

start XGB 100 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_ExcludeCaFA/WUSTL_Input100/x_test_adv_ZOO_Def1.npy not found
start XGB 100 CaFA
Accuracy: 0.08973334063406888

macro
Precision: 0.2323822825555685
Recall: 0.657020406018676
F1 Score: 0.08239078986523739

weighted
Precision: 0.8753750097099915
Recall: 0.08973334063406888
F1 Score: 0.04426909564038427

start XGB 100 SINIFGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9878678704748676

macro
Precision: 0.5005241414455674
Recall: 0.41624236487645283
F1 Score: 0.25448614143989146

weighted
Precision: 0.9976403991864039
Recall: 0.9878678704748676
F1 Score: 0.9913204069758281

start XGB 100 VNIFGSM


In [ ]:
print("a")
# torch.save(model.state_dict(), "./Adversarial_Training_pgd.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# # Iterate over epsilon values
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")